# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/lamo/Desktop/Cources/UdacityaDataEngineering/4. Non-Relational Databases/project/DataModelingWithApacheCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)


8056


In [5]:
# creating a smaller event data csv file called event_datafile_full csv
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))



In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821




The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS UdacitySparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacitysparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_sessions "
query = query + "(sessionId int, item_in_session int, artist_name text, song text, song_length float, PRIMARY KEY (sessionId, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO music_sessions (sessionId, item_in_session, artist_name, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9],float(line[5])))

##### In the first query we are interested in retrieving the artist, song and song length from music logs, we chose the (session id and item in session) as a unique primary key

In [12]:
#QUERY 1
session.row_factory = pandas_factory

query = "SELECT artist_name, song, song_length FROM music_sessions where sessionId=338 and item_in_session=4"

q1 = session.execute(query)._current_rows 
q1


,artist_name,song,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [13]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_sessions "
query = query + "(userId int ,sessionId int, item_in_session int, artist_name text, song text,\
user_first_name text, user_last_name text, PRIMARY KEY ((userId, sessionId), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [14]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_sessions (artist_name, song, item_in_session, user_first_name, user_last_name, userId, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s,%s,%s,%s)"
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))

##### In the second query we are interested in retrieving the artist song and user first and last name, we want our data sorted by item in session field, therefore we chose the (userid and session id) as unique primary key and (item in session) as a clustering column to sort by it

In [15]:
#QUERY 2
session.row_factory = pandas_factory

query = "SELECT artist_name, song, user_first_name, user_last_name FROM user_sessions\
    where userId=10 and sessionId=182"

    
q2 = session.execute(query)._current_rows 
q2

,artist_name,song,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [16]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + "(song text, userId int, user_first_name text, user_last_name text, PRIMARY KEY(song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

                    

In [17]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO user_by_song (userId, user_first_name, user_last_name, song)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

##### In the third query we are interested in getting songs listened by each user, retrieving user first and last name as well as the song, to make every primary key unique we partition by song and user id

In [18]:
#QUERY 3
session.row_factory = pandas_factory

query = "SELECT user_first_name, user_last_name, song FROM user_by_song\
    where song='All Hands Against His Own'"

q3 = session.execute(query)._current_rows 
q3

,user_first_name,user_last_name,song
0,Jacqueline,Lynch,All Hands Against His Own
1,Tegan,Levine,All Hands Against His Own
2,Sara,Johnson,All Hands Against His Own


### Drop Tables

In [19]:
query = "drop table music_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [20]:
query = "drop table user_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [21]:
query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()